# LE RÉSEAU

# Manipulation des paquets réseau

Le but de cet exercice est de voire quelques notions sur le réseau et d'apprendre à envoyer une requette "ping" sur un ordinateur, grâce à son adresse IP et son adresse MAC.
Il y a une grande différence entre ces deux adresses, disons que l'adresse **IP** est le nom d'une machine sur le web, et que l'adresse **MAC** est l'acronyme pour "media access control address", c'est une adresse électronique qui est propre au dispositif physique de transmission des données.

On va d'abord commencer par importer la librairie *scapy* qui contient un grand nombre d'outils pour étudier le réseau **TCP-IP**

- L'acronyme **TCP** vient de l'anglais "Transmission Control Protocol"
- L'acronyme **IP** vient de l'anglais "Internet Protocol"

In [2]:
from scapy.all import *

L'échange de paquets avec un serveur web est loin d'être simple, elle fait intervenir le **protocole** HTTP, le **handshake** TCP, l'**entête** IP, bref, nous allons au rester plus basique pour voire comment l'on envoie une requette à un appareil et comment on peut détecter des requettes venant qui sont faites à travers un reseau.

- Une **requette** est une demande polie, formulée dans les règles de l'art de s'entendre entre ordinateurs. N'imaginez pas comprendre le contenu d'une requette pour le moment, ce sont des formules qui utilisent un langage codé
- Un **protocole** est un ensemble de règles de communication
- Un **handshake** est une poignée de main, c'est la mise en relation de l'émeteur avec son récepteur on dit aussi entre le host et le client
- Une **entête** est l'information qui introduit le début d'un message IP

Commençons donc par créer et afficher une trame Éthernet dans l'interpréteur Scapy :



In [3]:
ma_trame = Ether()
ma_trame.show()

###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = b8:e8:56:34:af:08
  type      = 0x9000



Ici on peut décortiquer un message de la façon suivante : 
- `dst` est l'adresse MAC du destinataire

- `src` est l'adresse MAC de l'expéditeur

- `type` est la taille du paquet à envoyer 

La commande ping permet de savoir si un hôte, désigné par son adresse IP, existe. En version simplifiée, la commande *ping* consiste à envoyer un paquet "echo-request" à l'hôte et à dire si un paquet "echo-reply" a été renvoyé.

Forgeons (oui : "*forger*" est le terme consacré par les ingénieurs réseau) donc un paquet echo-request pour envoyer un ping à la machine dont d'adresse est donnée par la variable `dst`

In [4]:
mon_ping = mon_ping = Ether() / IP(dst='192.168.0.1') / ICMP()
mon_ping.show()

###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = b8:e8:56:34:af:08
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = icmp
     chksum    = None
     src       = 192.168.0.28
     dst       = 192.168.0.1
     \options   \
###[ ICMP ]### 
        type      = echo-request
        code      = 0
        chksum    = None
        id        = 0x0
        seq       = 0x0



Voyons maintenant si notre routeur va répondre à cela par un paquet echo-reply.

In [5]:
sendp(mon_ping)


Sent 1 packets.


Si le message "Sent 1 packets" apparait c'est qu'assurément l'envoi a pu se faire correctement, mais cela ne nous informe pas sur la réception. Pour envoyer et recevoir, il faut utiliser les fonctions `srp()`. Celui-ci renvoie deux objets : le premier contient les paquets émis et leurs réponses associées, l'autre contient les paquets sans réponse.

In [6]:
rep,non_rep = srp(mon_ping)

Begin emission:
Finished sending 1 packets.

Received 466 packets, got 1 answers, remaining 0 packets


On voit qu'on a eu une réponse, zéro échecs, et que notre réponse est un paquet ! Examinons-le 

In [7]:
rep.show()

0000 Ether / IP / ICMP 192.168.0.28 > 192.168.0.1 echo-request 0 ==> Ether / IP / ICMP 192.168.0.1 > 192.168.0.28 echo-reply 0 / Padding


Le résultat est un couple (tuple à deux valeurs). Pour afficher le paquet émis (notre ICMP echo-request), on fera donc `rep[0][0].show()`, et pour le paquet reçu en réponse : `rep[0][1].show()`

In [8]:
rep[0][0].show()

###[ Ethernet ]### 
  dst       = ac:84:c9:1f:ee:72
  src       = b8:e8:56:34:af:08
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = icmp
     chksum    = None
     src       = 192.168.0.28
     dst       = 192.168.0.1
     \options   \
###[ ICMP ]### 
        type      = echo-request
        code      = 0
        chksum    = None
        id        = 0x0
        seq       = 0x0



In [9]:
rep[0][1].show()

###[ Ethernet ]### 
  dst       = b8:e8:56:34:af:08
  src       = ac:84:c9:1f:ee:72
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 28
     id        = 10411
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = icmp
     chksum    = 0xd0c8
     src       = 192.168.0.1
     dst       = 192.168.0.28
     \options   \
###[ ICMP ]### 
        type      = echo-reply
        code      = 0
        chksum    = 0xffff
        id        = 0x0
        seq       = 0x0
###[ Padding ]### 
           load      = '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



Essayons maintenant d'envoyer un ping à un destinataire inconnu à l'adresse IP 10.1.0.201, normallement on ne devrait pas recevoir de réponse.

In [10]:
rep = sr1(IP(dst='10.1.0.201') / ICMP(), timeout=0.5)

Begin emission:
Finished sending 1 packets.

Received 193 packets, got 0 answers, remaining 1 packets


## Scan d'une plage d'adresse

L'avantage de python est de pouvoir automatiser une commande comme ping sur une plage d'adresses IP. On va prendre les 25 premières machines du réseau, on va attendre une seconde à chaque fois pour s'assurer que la communication passe bien. On notera que le dernier chiffre de l'adresse IP permet d'arroser une plage d'une centaine de machines avec des requettes.

In [16]:
adresses_machines = '192.168.0.1-100'
rep,non_rep = sr( IP(dst=adresses_machines) / ICMP() , timeout=2)
for elem in rep : # elem représente un couple (paquet émis, paquet reçu)
	if elem[1].type == 0 : # 0 <=> echo-reply
		print('{} a renvoyé un echo-reply '.format(elem[1].src))


Begin emission:
Finished sending 100 packets.

Received 1274 packets, got 5 answers, remaining 95 packets
192.168.0.1 a renvoyé un echo-reply 
192.168.0.10 a renvoyé un echo-reply 
192.168.0.11 a renvoyé un echo-reply 
192.168.0.25 a renvoyé un echo-reply 
192.168.0.38 a renvoyé un echo-reply 


## Utilisation d'un "mouchard" sur le réseau

On peut également utiliser `sniff` pour scanner l'activité du réseau, c'est ce que l'on appelle un 'mouchard' car cette fonction va nous informer sur toutes les sites consultés et il peut être très utile pour savoir qui est ce qui vous envoie des requettes indésirables. En effet le harcélement sur internet et plus répandu que l'on ne le pense, surtout quand il est invisible.

Notez bien qu'à chaque raffraichissement l'activité devrait changer, car le réseau est comme une mer en mouvement, transportant ça et là des requetes et des réponses...

In [17]:
pkt = sniff(count=3, filter='tcp', prn=Packet.summary)

Ether / IP / TCP 192.168.0.28:58770 > 13.32.215.252:https PA / Raw
Ether / IP / TCP 192.168.0.28:58770 > 13.32.215.252:https FA
Ether / IP / TCP 13.32.215.252:https > 192.168.0.28:58770 FA


In [15]:
# Si on veut en savoir un peu plus sur la dernière requette
pkt[2]

<Ether  dst=00:90:a9:e3:b8:bb src=b8:e8:56:34:af:08 type=0x800 |<IP  version=4 ihl=5 tos=0x20 len=552 id=0 flags=DF frag=0 ttl=64 proto=tcp chksum=0xb739 src=192.168.0.28 dst=192.168.0.10 |<TCP  sport=57912 dport=afpovertcp seq=4294824139 ack=3654483264 dataofs=5 reserved=0 flags=A window=32768 chksum=0x3d47 urgptr=0 |<Raw  load="\xa6-\x9e\xa5\xa7Cf\xe3\x97\x04\xe4\x03\xf5\xae\x9b\xe1\xe6\x9b\xe0\xad*\xf1$\xf1M\x82_[\x06\xc3\x802\x08\xa9|S\xf0o^\xbaq\xabi\xbe,\xb4\x942\x9d\xf6\xd6N1\xef\x9c\xd5_\x08h\xf6\x1a:\x9bG-;\xa3\x1d\xc5\xdb<\xd76-\xce\x11M\x1d\xb8i\xfbH\xb5s\xac\xf8\x83\xf0\xb7\xc0\x97\xbak^\xfc(\xf8]uo C$\xda\x94\xf0\x00\x81z\x9e\x84\x9a\xf2\xfd/V\x16\x17\xa7\xc8+$i.\xc7\x0b\xeb\x9e\x7f*\xfa\xf3\xe14\x7f\xf0\x91\xfc,\x9a\xde\xff\x00^\x82\x04Eu\x96\xdeG\x19\t\xb7\xff\x00\xd7_3\xfcL\xf8]\xa2x\x1f\xc4SA\xe1\xbdB\x16\xb6\x9eV\x95Q\x1b\x80I\xcei\xe2\xa1N\xae\x19Inq\xe1\xea\xba8\xcdv>\x82\xfd\x97\xfe2|>\xf8\x7f\x13\xc1\xe3\xab\xabht\xfb\x98\xbec,A\x8e\xef\xcb'\xbf\x15\x0f\xed\x01\xf

C'est somme toute incomprehensible pour le commun des mortels, mais rassurez vous les machines comprennent très bien qu'il s'agit d'une requette de sauvegarde sur un disque externe !